# Library import 

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import logging
import numpy as np
import re
import tqdm
from threading import *
import queue
import time

# Target links

In [2]:
start=323000
end = 327000
link_list=[]
for i in range(start,end):
    link_list.append('http://www.newsmth.net/nForum/article/Intern/'+str(i)+'?ajax' )

# Scrapy Funtion

## Process extends Thread Class

In [3]:
class ShuimuSpider(Thread):
    def __init__(self,url,q):
        super(ShuimuSpider,self).__init__()
        self.url=url
        self.q=q
        
    def run(self):
        self.parse_page()
        
    def send_request(self,url):
        try:
            html = requests.get(url,timeout=2)
        except Exception as e:
            print("fail at %s" % url)
        else:
            return html
    def parse_page(self):
        response = self.send_request(self.url)
        if response.status_code==200:
            response_bs=response.content
            try:
                bs = BeautifulSoup(response_bs,'html.parser')


                text=''
                store_data={
                    'time':[],
                    'title':[],
                    'detail':[]
                }
                text = bs.p.text
                # .p 对应html中的<p>, 观察帖子不难发现，几乎所有的招聘正文都在帖子的<p>内
                # .text 对应输出文本

                #分别通过特殊字符串的位置给标题、时间、正文定位
                #position for title:
                Title_Position1=text.find('标  题: ')+len('标  题: ')
                Title_Position2=text.find('  发信站: 水木社区')
                Title = text[Title_Position1:Title_Position2]
                #text[x:y]进行切片

                #position for time:
                Time_Position1=text.find('  发信站: 水木社区')+len('  发信站: 水木社区')
                Time_Position2=text.find('), 站内')+1
                Time=text[Time_Position1:Time_Position2]

                #position for detail
                Detail_Position1=text.find('), 站内')+len('), 站内')+4
                Detail_Position2=text.find('※ 来源:·水木社区')
                Detail=text[Detail_Position1:Detail_Position2]

                store_data['time'].append(Time)
                store_data['title'].append(Title)
                store_data['detail'].append(Detail)
                #字典赋值
                #运行成功，index序列加一

                self.q.put(store_data)
            except:
                return
        else:
            return
            

## Scrapy function with multi-threading

In [4]:
def Fast_Scrapy(x,y,store_data):
    start = time.time()
    q= queue.Queue(0)

    Thread_list=[]
    for url in link_list[x:y]:
        p = ShuimuSpider(url,q)
        p.start()
        Thread_list.append(p)

    for i in Thread_list:
        i.join()

    while not q.empty():
        data = q.get()

        store_data['time'].append(data['time'])
        store_data['title'].append(data['title'])
        store_data['detail'].append(data['detail'])
    end = time.time()
    print('当前Process + Queue多线程爬虫的总时间为：', end-start)

## Scrapy RUN

In [5]:
def Scrapy():
    store_data={
        'time':[],
        'title':[],
        'detail':[]
    }
    if end-start>150:
        period = np.arange(0,end-start,150)

        for i in tqdm.tqdm(period):
            Fast_Scrapy(i,i+150,store_data)
        Fast_Scrapy(i,end,store_data)
    else:
        Fast_Scrapy(start,end,store_data)
        
    df = pd.DataFrame(store_data)
    df.to_csv('../dataset/'+str(start)+'-'+str(end)+'.csv')
    return df

In [6]:
%%time
df = Scrapy()

  4%|███                                                                                | 1/27 [00:04<01:46,  4.09s/it]

当前Process + Queue多线程爬虫的总时间为： 4.093417406082153


  7%|██████▏                                                                            | 2/27 [00:08<01:42,  4.08s/it]

当前Process + Queue多线程爬虫的总时间为： 4.068395376205444


 11%|█████████▏                                                                         | 3/27 [00:12<01:36,  4.01s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▉   | 26/27 [01:39<00:04,  4.04s/it]

当前Process + Queue多线程爬虫的总时间为： 4.091559886932373


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [01:42<00:00,  3.78s/it]

当前Process + Queue多线程爬虫的总时间为： 2.910522937774658


当前Process + Queue多线程爬虫的总时间为： 2.856959819793701
Wall time: 1min 45s


# Data Cleaning

## Remove '[]'

In [7]:
# remove '[]'
for col in ['time','title','detail']:
    df[col]=df[col].apply(lambda x:x[0])

## Transfer to pd.datetime

In [8]:
# transfer to pd.datetime
df['time']=(df['time'].str[2:12]+df['time'].str[21:-1])
df['time']=pd.to_datetime(df['time'],errors='coerce')
df['time']=df['time'].fillna(method='ffill')

## remove data with extremely short title

In [9]:
df = df[df['title'].str.len()>5]

## 去重

In [10]:
df = df[~df['title'].duplicated()]
df.reset_index(drop=True,inplace=True)

## Remove datas without a delivering email

In [85]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex = re.compile(pattern,flags=re.IGNORECASE)
df['Email']='NaN'
for i in range(len(df)):
    try:
        df.iloc[i,3]=regex.findall(df['detail'][i])
    except:
        continue
df.replace(to_replace='NaN',value = np.nan,regex=True,inplace =True)
df.dropna(axis=0,how='any',inplace =True)
df.reset_index(drop=True,inplace=True)

## output as text file

In [89]:
def output(df):
    text=''
    for i in range(len(df)):
        text += str(df.index[i])+' '
        text += df['title'][i]+'\n'
        text += df['detail'][i]+'\n'
        text += df['Email'][i]+'\n\n\n'
    
    with open(str(start)+'-'+str(end)+'.txt','w',encoding='utf-8') as f:
        f.write(text)

# Classify

## Load Classify Model

In [33]:
import joblib
clf_model=joblib.load('../models/Classification_model_5_n_estimator=100_pca=400.joblib')

## Predict function

In [19]:
mark = ['产品经理', '人力资源', '券商投行基金', '前端', '咨询', '外语实习生', '大数据', '投资实习生',
       '数据分析', '数据挖掘', '测试工程师', '研发开发', '算法实习', '行业研究', '行政', '设计',
       '财务会计审计税务', '运营实习', '量化交易', '销售']

In [26]:
import jieba_fast

stopwords = pd.read_csv('../stopwords/baidu_stopwords.txt',
                        quoting=3,sep='\t',names=['stopword'],encoding='utf-8')
stopwords = stopwords['stopword'].values

def pre_input(input):
    sentences=[]
    segs=jieba_fast.cut(input,cut_all=False)
    segs = list(filter(lambda x:x.strip(),segs))
    segs=list(filter(lambda x:((x not in stopwords) and x != 'xa0' ),segs))
    sentences.append(str([x for x in segs]))
    return sentences

def predict_df(x):
    return mark[clf_model.predict(pre_input(x))[0]]

## predict for dataframe

In [15]:
df['category']=np.zeros(len(df))

In [27]:
df['category']=df['detail'].apply(lambda x:predict_df(x))

In [32]:
df[20:40]

,time,title,detail,category
20,2019-11-26,用友网络科技股份有限公司法务部招收实习生,岗位职责： 1.起草、审核合同及各类法律文件，制定标准合同文本，参与与合同相关的业务设计、...,券商投行基金
21,2019-11-26,【实习-汇安基金】量化实习生（有留用机会）,单位简介： 汇安基金管理有限责任公司，2016年4月19日获中国证监会批准，并在上海注册成...,券商投行基金
22,2019-11-26,【实习】【上海】方正证券研究所金融工程组招聘量化实习生,【实习】【上海】方正证券研究所金融工程组招聘量化实习生 【工作职责】 1、协助开发资产配...,量化交易
23,2019-11-26,【招聘】中国新闻出版研究院,中国新闻出版研究院国际出版研究室 招聘小语种实习生 中国新闻出版研究院是我国唯一...,大数据
24,2019-11-26,【实习】【北京】悟语学堂招聘传播/推广岗-坐标骏豪中央公园,坐标：朝阳公园地铁边上骏豪中央公园 团队优势：弹性工作制、最火热最有价值的教育行业、行业大...,运营实习
25,2019-11-26,【微软】【北京】【实习】前端实习生,热爱前端开发的同学，可以投简历过来yanhu@microsoft.com 表现出色可以争取...,研发开发
26,2019-11-26,【实习】【七鑫易维】 深度学习实习生,北京七鑫易维信息技术有限公司（www.7invensun.com）是致力于机器视觉和人工智...,算法实习
27,2019-11-26,【实习】【七鑫易维】 图像算法实习生,北京七鑫易维信息技术有限公司（www.7invensun.com）是致力于机器视觉和人工智...,算法实习
28,2019-11-26,【实习】【NVIDIA】英伟达2020实习季，期待你的加入！,又是一年校招季的结束，当2020届的同学已经找到心仪工作的同时，2021届的同学们是否也蠢蠢...,研发开发
29,2019-11-26,深圳Using.ai 招聘算法实习,关于我们 贝尔信息是一家具备优秀且可商业化的机器学习、视觉技术和一体式硬件研发能力的高...,算法实习
